In [ ]:
! python -m pip install gurobipy
! python -m pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 62.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Lettura file CSV
"""
Leggo il file 'frighi.csv' in un DataFrame pandas.
Ogni riga del file rappresenta un fornitore (o una riga speciale 'offerta').
Ogni colonna rappresenta una destinazione (o una colonna speciale 'domanda').
"""
df = pd.read_csv("data/frighi.csv")

"""
Ricavo il nome della prima colonna del DataFrame (ad esempio 'i'),
che contiene le etichette delle righe (es. fornitori, 'offerta', …).
La salvo in idx per poterla usare come indice.
"""
idx = df.columns.tolist()[0]

"""
Imposto la colonna identificata in idx come indice di riga del DataFrame.
In questo modo posso accedere alle righe per nome (es. df.loc['offerta']).
"""
df = df.set_index(f'{idx}')

# Definizione insiemi
"""
I = insieme degli stabilimenti (righe di C)
J = insieme dei magazzini (colonne di C)
"""
I = df.drop(columns="domanda").columns.tolist()
J = df.drop(index="offerta").index.tolist()


# Definizione dei parametri
"""
Estraggo il vettore riga 'offerta' (insieme O) e la converto in dizionario:
- chiavi = nomi delle colonne (cioè gli stabilimenti)
- valori =  offerta di ciascun stabilimento.
O = {
  'a' : 50,
  'b' : 70, ...
}
"""
O = df.loc["offerta"].to_dict()

"""
Estraggo il vettore colonna 'domanda' (insieme D) e la converto in dizionario:
- chiavi = nomi delle righe (cioè le destinazioni, ovvero i magazzini)
- valori = domanda per ciascun magazzino.
D = {
  '1' : 10,
  '2' : 60, ...
}
"""
D = df["domanda"].to_dict()

"""
Estraggo la matrice dei costi di trasporto tra gli stabilimenti e i magazzini
C[i][j] = costo di trasporto dallo stabilimento i al magazzino j.
"""
C  = df.drop(index="offerta").drop(columns="domanda")


print(f"Insieme degli stabilimenti (I): {I}")
print(f"Insieme dei magazzini (J): {J}")
print(f"Insieme delle offerte (O): {O}")
print(f"Insieme delle domande (D): {D}")
print(f"Matrice dei costi (C):\n{C}")


# Creazione modello
m = gp.Model("trasporto_frighi")
m.setParam("OutputFlag", 1)

# Creazione variabili x_ij
x = m.addVars(I, J, vtype=GRB.INTEGER, lb=0, name="x")

# Creazione vincoli
for i in I:
    m.addConstr(gp.quicksum(C.loc[j, i] * x[i, j] for j in J) <= O[i], name=f"origine[{i}]")

for j in J:
    m.addConstr(gp.quicksum(C.loc[j, i] * x[i, j] for i in I) >= D[j], name=f"destinazione[{j}]")

# Definizione funzione obiettivo
m.setObjective(gp.quicksum(C[i][j] * x[i, j] for i in I for j in J), GRB.MINIMIZE)


# Ottimizzazione
m.optimize()

# Gestione stato del modello
status  = m.status

if status == GRB.OPTIMAL:
    print("Soluzione ottima trovata.")
elif status == GRB.SUBOPTIMAL:
    print("Soluzione ammissibile trovata ma non garantita ottima (SUBOPTIMAL).")
elif status in (GRB.TIME_LIMIT, GRB.NODE_LIMIT, GRB.ITERATION_LIMIT, GRB.SOLUTION_LIMIT):
    # Limiti raggiunti: mostro il meglio disponibile
    limits = {
        GRB.TIME_LIMIT: "TIME_LIMIT",
        GRB.NODE_LIMIT: "NODE_LIMIT",
        GRB.ITERATION_LIMIT: "ITERATION_LIMIT",
        GRB.SOLUTION_LIMIT: "SOLUTION_LIMIT",
    }
    print(f"Arresto per limite: {limits.get(status, 'LIMIT')}.")
elif status == GRB.UNBOUNDED:
    print("Modello non limitato (UNBOUNDED).")
elif status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE):
    print("Modello inammissibile o inammissibile/non limitato (INF_OR_UNBD/INFEASIBLE).")
elif status in (GRB.INTERRUPTED, GRB.NUMERIC):
    c = "INTERRUPTED" if status == GRB.INTERRUPTED else "NUMERIC"
    print(f"Risoluzione terminata: {c}.")
else:
    print("Stato non gestito esplicitamente. Consulta la documentazione Gurobi per dettagli.")

# Visualizzazione risultati
if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
    print(f"Costo totale ottimo: {m.objVal:.0f}")

    # Riepilogo spedizioni positive
    print("\nSpedizioni (i -> j : quantità):")
    for i in I:
        for j in J:
            val = x[i, j].X
            if val > 1e-6:
                print(f"{i} -> {j} : {val:.0f}")

    # Controllo bilanciamento per ogni origine e destinazione
    print("\nUtilizzo offerta:")
    for i in I:
        sped = sum(x[i, j].X for j in J)
        print(f"{i}: spedito {sped:.0f} / offerta {O[i]}")

    print("\nDomanda soddisfatta:")
    for j in J:
        ricev = sum(x[i, j].X for i in I)
        print(f"{j}: ricevuto {ricev:.0f} / domanda {D[j]}")

Insieme degli stabilimenti (I): ['a', 'b', 'c']
Insieme dei magazzini (J): ['1', '2', '3', '4']
Insieme delle offerte (O): {'a': 50.0, 'b': 70.0, 'c': 20.0, 'domanda': nan}
Insieme delle domande (D): {'1': 10.0, '2': 60.0, '3': 30.0, '4': 40.0, 'offerta': nan}
Matrice dei costi (C):
           a  b  c
magazzini         
1          6  2  2
2          8  3  4
3          3  1  6
4          4  3  5
Set parameter Username
Academic license - for non-commercial use only - expires 2026-06-10
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x8cc9535c
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1